In [10]:
import polars as pl     # requires installing polars first
import pyarrow          # requires installing pyarrow first

import re

In [11]:
# CodeBook
cb = pl.read_csv("data/codebooks/PreFer_codebook.csv").to_pandas()

In [12]:
cb[cb["var_name"] == "cf08a003"]

,var_name,var_label,values_cat,labels_cat,unique_values_n,n_missing,prop_missing,type_var,note,year,survey,dataset
41,cf08a003,Gender respondent,1; 2,male; female,2.0,28,0.003938,categorical,,2008.0,Family & Household,PreFer_train_data.csv


In [171]:
df = pl.read_csv("data/training_data/PreFer_train_data.csv",
                     infer_schema_length=7418)

In [172]:
df.head()

nomem_encr,outcome_available,cf08a_m,cf09b_m,cf10c_m,cf11d_m,cf12e_m,cf13f_m,cf14g_m,cf15h_m,cf16i_m,cf17j_m,cf18k_m,cf19l_m,cf20m_m,cf08a001,cf09b001,cf10c001,cf11d001,cf12e001,cf13f001,cf14g001,cf15h001,cf16i001,cf17j001,cf18k001,cf19l001,cf20m001,cf08a002,cf09b002,cf10c002,cf11d002,cf12e002,cf13f002,cf14g002,cf15h002,cf16i002,…,sted_2012,sted_2013,sted_2014,sted_2015,sted_2016,sted_2017,sted_2018,sted_2019,sted_2020,woning_2007,woning_2008,woning_2009,woning_2010,woning_2011,woning_2012,woning_2013,woning_2014,woning_2015,woning_2016,woning_2017,woning_2018,woning_2019,woning_2020,woonvorm_2007,woonvorm_2008,woonvorm_2009,woonvorm_2010,woonvorm_2011,woonvorm_2012,woonvorm_2013,woonvorm_2014,woonvorm_2015,woonvorm_2016,woonvorm_2017,woonvorm_2018,woonvorm_2019,woonvorm_2020
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
712619,0,200803,200904,null,null,201203,201303,null,null,null,null,null,null,null,16,16,null,null,16,16,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,2,4,4,4,4,null,null,null,null,2,2,2,2,2,2,2,2,2,2,null,null,null,null,3,4,4,4,4,4,4,4,4,4,null,null,null,null
706448,0,200803,200903,201003,201103,201203,201303,201403,201509,201609,201709,null,null,null,16,16,1,1,1,1,1,1,1,1,null,null,null,null,null,2005,2005,2005,2005,2005,2005,2005,…,3,3,3,3,3,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3
729145,0,200803,200903,201003,201103,201203,null,201403,201509,201609,201709,null,201909,202009,16,16,16,16,1,null,1,2,2,2,null,2,2,null,null,null,null,2007,null,2007,2009,2009,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,3,3,3,3,3,3,3,4,4,4,3,3,3,3
729424,0,200803,200903,null,null,null,null,null,null,null,null,null,null,null,16,16,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,1,1,2,2,null,null,null,null,null,null,null,null,null,null,2,2,2,2,null,null,null,null,null,null,null,null,null,null
715619,1,200803,200903,201003,201103,201203,201303,201403,201509,201609,201709,201809,201909,202009,16,16,16,16,16,16,16,16,16,16,16,16,16,null,null,null,null,null,null,null,null,null,…,2,2,2,2,2,2,3,3,3,2,2,2,2,2,2,2,2,2,2,2,1,1,1,4,4,4,4,4,4,4,1,2,2,2,2,2,2


In [173]:
YEAR = "08"
pattern = re.compile(r"^[A-Za-z]{2}%s[A-Za-z][0-9]{3}$" %YEAR)
filtered_columns = [name for name in df.columns if pattern.match(name)]
filtered_columns

['cf08a001',
 'cf08a002',
 'cf08a003',
 'cf08a004',
 'cf08a005',
 'cf08a006',
 'cf08a007',
 'cf08a008',
 'cf08a009',
 'cf08a010',
 'cf08a011',
 'cf08a012',
 'cf08a013',
 'cf08a014',
 'cf08a015',
 'cf08a016',
 'cf08a017',
 'cf08a018',
 'cf08a019',
 'cf08a020',
 'cf08a021',
 'cf08a022',
 'cf08a023',
 'cf08a024',
 'cf08a025',
 'cf08a026',
 'cf08a027',
 'cf08a028',
 'cf08a029',
 'cf08a030',
 'cf08a031',
 'cf08a032',
 'cf08a033',
 'cf08a034',
 'cf08a035',
 'cf08a036',
 'cf08a037',
 'cf08a038',
 'cf08a039',
 'cf08a040',
 'cf08a041',
 'cf08a042',
 'cf08a043',
 'cf08a044',
 'cf08a045',
 'cf08a046',
 'cf08a047',
 'cf08a048',
 'cf08a049',
 'cf08a050',
 'cf08a051',
 'cf08a052',
 'cf08a053',
 'cf08a054',
 'cf08a055',
 'cf08a056',
 'cf08a057',
 'cf08a058',
 'cf08a059',
 'cf08a060',
 'cf08a061',
 'cf08a062',
 'cf08a063',
 'cf08a064',
 'cf08a065',
 'cf08a066',
 'cf08a067',
 'cf08a068',
 'cf08a069',
 'cf08a070',
 'cf08a071',
 'cf08a072',
 'cf08a073',
 'cf08a074',
 'cf08a075',
 'cf08a076',
 'cf08a077',

In [261]:
df_simple = df[filtered_columns[:40]]
row_check = df_simple.select(pl.all().is_null()).sum_horizontal() < 30
df_simple = df_simple.filter(row_check)#.fill_nan(-1).fill_null(-1)
df_simple = df_simple.with_columns( target = df_simple.select(pl.all().is_null()).sum_horizontal())
df_simple.sample(n=10)

cf08a001,cf08a002,cf08a003,cf08a004,cf08a005,cf08a006,cf08a007,cf08a008,cf08a009,cf08a010,cf08a011,cf08a012,cf08a013,cf08a014,cf08a015,cf08a016,cf08a017,cf08a018,cf08a019,cf08a020,cf08a021,cf08a022,cf08a023,cf08a024,cf08a025,cf08a026,cf08a027,cf08a028,cf08a029,cf08a030,cf08a031,cf08a032,cf08a033,cf08a034,cf08a035,cf08a036,cf08a037,cf08a038,cf08a039,cf08a040,target
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,u32
16,null,1,28,1951,null,2,2007,1955,null,1,null,2,null,null,2,null,null,1,null,null,6,6,1,1,1982,5,2002,2004,1,2004,2,2,null,1,1,2006,null,null,null,14
16,null,1,20,1957,null,1,null,1959,null,1,null,2,null,null,null,1,null,null,4,4,4,4,1,1,1987,1,2005,2005,1,2005,2,2,null,1,2,2005,2007,null,null,13
16,null,1,32,1953,null,1,null,1954,null,2,1995,2,null,1,null,null,1,null,4,4,null,null,2,null,null,null,null,null,null,null,null,2,null,2,null,null,null,null,null,24
16,null,2,29,1954,null,1,null,1956,null,1,null,1,2,1,2,null,1,1,1,3,1,1,1,1,1978,1,1997,1999,2,null,1,2,null,2,null,null,null,null,null,13
16,null,1,16,1957,null,1,null,1960,null,1,null,2,null,null,null,4,null,null,7,null,7,null,2,null,null,null,null,null,null,null,null,2,null,2,null,null,null,null,null,26
16,null,1,30,1949,null,1,null,1952,null,1,null,2,null,null,null,1,null,null,5,5,5,5,1,2,1986,1,2004,null,2,null,2,2,null,2,null,null,null,null,null,18
16,null,2,27,1956,null,1,null,1954,null,1,null,2,null,null,null,1,null,null,6,5,6,6,1,1,1977,1,1999,2005,2,null,1,2,null,2,null,null,null,null,null,17
16,null,1,16,1959,null,1,null,1959,null,1,null,2,null,null,null,4,null,null,7,null,7,null,2,null,null,null,null,null,null,null,null,2,null,2,null,null,null,null,null,26
16,null,2,21,1961,null,1,null,1962,null,1,null,2,null,null,null,4,null,null,7,null,7,null,1,2,1986,1,2005,null,2,null,1,2,null,2,null,null,null,null,null,20


In [262]:
import torch_frame
from torch_frame.data import Dataset, DataLoader
import torch
if torch.backends.mps.is_available():
    device = torch.device("mps")
    x = torch.ones(1, device=device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [264]:
stype_dict = {}

for col in df_simple.columns:
    try:
        stype = cb[cb["var_name"] == col]['type_var'].values[0]
    except:
        stype = 'numeric'

    if col == 'nomem_encr':
        continue
    print(col, stype)

    if stype == 'numeric':
        stype_dict[col] = torch_frame.numerical
    elif stype == 'categorical':
        stype_dict[col] = torch_frame.categorical

cf08a001 categorical
cf08a002 numeric
cf08a003 categorical
cf08a004 numeric
cf08a005 numeric
cf08a006 categorical
cf08a007 categorical
cf08a008 numeric
cf08a009 numeric
cf08a010 categorical
cf08a011 categorical
cf08a012 numeric
cf08a013 categorical
cf08a014 numeric
cf08a015 categorical
cf08a016 categorical
cf08a017 categorical
cf08a018 categorical
cf08a019 categorical
cf08a020 categorical
cf08a021 categorical
cf08a022 categorical
cf08a023 categorical
cf08a024 categorical
cf08a025 categorical
cf08a026 numeric
cf08a027 categorical
cf08a028 numeric
cf08a029 numeric
cf08a030 categorical
cf08a031 numeric
cf08a032 categorical
cf08a033 categorical
cf08a034 categorical
cf08a035 categorical
cf08a036 numeric
cf08a037 numeric
cf08a038 numeric
cf08a039 numeric
cf08a040 numeric
target numeric


In [265]:
dataset = Dataset(df=df_simple.to_pandas(),
                  col_to_stype= stype_dict,
                  target_col='target')
dataset.materialize()
dataset = dataset.shuffle()
# Split ratio is set to 80% / 10% / 10% (no clear mentioning of split in the
# original TabNet paper)
train_dataset, test_dataset = dataset[:0.7], dataset[0.7:]

# Set up data loaders
train_tensor_frame = train_dataset.tensor_frame
test_tensor_frame = test_dataset.tensor_frame
train_loader = DataLoader(train_tensor_frame, batch_size=4,
                          shuffle=True)
test_loader = DataLoader(test_tensor_frame, batch_size=4)


In [266]:
print(len(dataset))

1567


In [267]:
from typing import Any, Dict, List

import numpy as np

from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch_frame
from torch_frame import TensorFrame, stype
from torch_frame.data.stats import StatType
from torch_frame.nn.conv import TabTransformerConv
from torch_frame.nn.encoder import (
    EmbeddingEncoder,
    LinearEncoder,
    StypeWiseFeatureEncoder)

class DeepTransform(nn.Module):
    def __init__(
        self,
        channels: int,
        out_channels: int,
        num_layers: int,
        num_heads: int,
        col_stats: Dict[str, Dict[StatType, Any]],
        col_names_dict: Dict[torch_frame.stype, List[str]],
    ):
        super().__init__()
        self.encoder = StypeWiseFeatureEncoder(
            out_channels=channels,
            col_stats=col_stats,
            col_names_dict=col_names_dict,
            stype_encoder_dict={
                stype.categorical: EmbeddingEncoder(),
                stype.numerical: LinearEncoder()
            },
        )
        
        self.tab_transformer_convs = nn.ModuleList([
            TabTransformerConv(
                channels=channels,
                num_heads=num_heads,
                attn_dropout=0.1,
                ffn_dropout=0.1
            ) for _ in range(num_layers)
        ])
        self.decoder = nn.Linear(channels, out_channels)

    def forward(self, tf: TensorFrame) -> Tensor:
        x, _ = self.encoder(tf)
        for tab_transformer_conv in self.tab_transformer_convs:
            x = tab_transformer_conv(x)
        out = self.decoder(x.mean(dim=1))
        return out
# Predict Corruptions

encoder = DeepTransform(channels=32,
                        out_channels=1,
                        num_layers=2,
                        num_heads=8,
                        col_stats=train_dataset.col_stats,
                        col_names_dict=train_dataset.tensor_frame.col_names_dict).to(device)

In [268]:
optimizer = torch.optim.Adam(encoder.parameters())

for epoch in range(30):
    loss_metric = list()
    for tf in train_loader:
        tf = tf.to(device)
        pred = encoder(tf)
        loss = F.smooth_l1_loss(pred, tf.y)
        loss_metric.append(loss.detach().cpu().numpy())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(epoch, np.mean(loss_metric))


/var/folders/pk/3vzybg253k1d3n7qzxkts_2c0000gn/T/ipykernel_6235/779718972.py:8: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.smooth_l1_loss(pred, tf.y)
/var/folders/pk/3vzybg253k1d3n7qzxkts_2c0000gn/T/ipykernel_6235/779718972.py:8: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.smooth_l1_loss(pred, tf.y)


0 4.6023555
1 3.6078134
2 3.6486893
3 3.5386946
4 3.5218205
5 3.518623
6 3.5418563
7 3.4859295
8 3.5078475
9 3.5995674
10 3.5241425
11 3.5906608
12 3.481145
13 3.4751368
14 3.51034
15 3.5160975
16 3.504066
17 3.4913816
18 3.5025015
19 3.476742
20 3.4252322
21 3.4748297
22 3.439675
23 3.4918
24 3.4798691
25 3.5061376
26 3.4492235
27 3.4760983
28 3.4225712
29 3.4748304


In [269]:
encoder.eval()
correct = 0
test_loss = list()
for tf in test_loader:
    tf = tf.to(device)
    pred = encoder(tf)
    test_loss.append (torch.abs(tf.y - pred).detach().cpu().numpy())
acc = np.mean(test_loss)
print(f'Accuracy: {acc:.4f}')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (118,) + inhomogeneous part.

In [273]:
pred

tensor([[17.5639],
        [20.4563]], device='mps:0', grad_fn=<LinearBackward0>)

In [204]:
out

tensor([[-0.1649,  0.1629, -0.1667, -0.1146,  0.0091,  0.2028,  0.2592, -0.0893,
          0.1435, -0.1267,  0.1180, -0.1580,  0.1417,  0.1024, -0.0005,  0.0576,
          0.1325, -0.1485,  0.2000,  0.1296, -0.2248,  0.1822, -0.1108,  0.0778,
         -0.0879, -0.2285, -0.1510, -0.0922, -0.1953,  0.1255,  0.1357,  0.0043],
        [-0.1723,  0.1555, -0.1686, -0.1001,  0.0007,  0.2104,  0.2450, -0.1024,
          0.1435, -0.1061,  0.1091, -0.1394,  0.1294,  0.1189,  0.0079,  0.0407,
          0.1525, -0.1538,  0.1968,  0.1317, -0.2175,  0.1839, -0.0981,  0.0721,
         -0.0802, -0.2341, -0.1557, -0.0849, -0.1727,  0.1196,  0.1395,  0.0055],
        [-0.1740,  0.1639, -0.1611, -0.1032,  0.0107,  0.2053,  0.2571, -0.0832,
          0.1463, -0.1196,  0.1184, -0.1617,  0.1400,  0.0970,  0.0077,  0.0473,
          0.1340, -0.1591,  0.1968,  0.1281, -0.2120,  0.1788, -0.1081,  0.0758,
         -0.0840, -0.2360, -0.1612, -0.0877, -0.1793,  0.1157,  0.1319,  0.0144],
        [-0.1621,  0.1701

torch.Size([4, 32])